<a href="https://colab.research.google.com/github/JohnYCLam/Statistics/blob/main/Summary_of_Statistical_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Illustration and summary of different statistical testing.

*   Construction of Confident Interval
*   hypothesis testing (Z Test, T Test, $\chi^2$ Test, Wald's Test, Likelihood Ratio Test, Testing implicit hypothesis)
*   normality test/ goodness of fit testing (Jarque Bera Test, D'Agostino and Pearson Test, Kolmogorov-Smirnov test, Kolmogorov-Lilliefors test, Quantile-Quantile (QQ) plots, $\chi^2$ goodness-of-fit test)


Construction of Confidence Interval


In [1]:
import numpy as np
import scipy.stats as stats

In [ ]:
#Construct a confidence interval of level 95% for the mean p of Bernoulli(p)
alpha = 0.05
q_alpha2 = stats.norm.ppf(1-alpha/2)

#Generate synthetic data
n = 1000
X = np.random.binomial(1, 0.5, n)
X_avg = X.mean()

#Confidence interval Methnod 1: replace p(1-p) by 1/4
I = [X_avg - q_alpha2/(2*n**0.5), X_avg + q_alpha2/(2*n**0.5)]
print(I)

#Confidence interval Method 2: replace p by X_avg
I = [X_avg - q_alpha2 * (X_avg*(1 - X_avg)/n)**0.5, X_avg + q_alpha2 * (X_avg*(1 - X_avg)/n)**0.5]
print(I)

#Confidence interval Method 3: Calculate the exact range of p
a = n + q_alpha2**2
b = - (2*n*X_avg + q_alpha2**2)
c = n*X_avg**2
I = [(-b - (b**2 - 4*a*c)**0.5)/(2*a), (-b + (b**2 - 4*a*c)**0.5)/(2*a)]
print(I)

[0.4670102483847719, 0.5289897516152281]
[0.4670104963037765, 0.5289895036962234]
[0.4670775003823489, 0.5289378066515933]


In [ ]:
#Meaning of Confidence interval
I = []
trials = 1000
n = 100
p = 0.5
count = 0

for _ in range(trials):
  X = np.random.binomial(1, p, n)
  X_avg = X.mean()
  lower = X_avg - q_alpha2/(2*n**0.5)
  upper = X_avg + q_alpha2/(2*n**0.5)
  I.append([lower, upper])
  if lower > p or upper < p:
    count += 1
print(f'Out of the {trials} constructed confidence intervals at 95% confidence level,\n{count} of them do not contain p ({p})')

Out of the 1000 constructed confidence intervals at 95% confidence level,
59 of them do not contain p (0.5)


Z Test: Underlying distribution is (asymptotically) normal AND the population variance is known. Then Z test is equivalent to testing whether the mean lies in the confidence interval.

In [78]:
#Generate synthetic data
n = 100
p = 0.6
X = np.random.binomial(1, p, n)
X_avg = X.mean()
u = p
sigma_square = p*(1-p)

test_statistic = abs(n**0.5 * (X_avg - u)/(sigma_square)**0.5)
test_statistic

1.6329931618554536

In [79]:
alpha = 0.05
q_alpha = stats.norm.ppf(1-alpha/2)
if test_statistic > q_alpha:
  print("Reject H_0")
else:
  print("Failed to reject H_0")

Failed to reject H_0


In [86]:
#Repeat the test for 100 times
#Expect to reject H_0 about 5 times
trials = 100
alpha = 0.05
q_alpha = stats.norm.ppf(1-alpha/2)
u = p
sigma_square = p*(1-p)
results = []

for _ in range(trials):
  X = np.random.binomial(1, p, n)
  X_avg = X.mean()
  test_statistic = abs(n**0.5 * (X_avg - u)/(sigma_square)**0.5)
  results.append(test_statistic > q_alpha)

print(f'After {trials} trials, the null hypothesis is rejected {sum(results)} times at 95% confidence.')

After 100 trials, the null hypothesis is rejected 2 times at 95% confidence.


T Test: when we have unknown population variance. Assumption: X_i ~ N(u, sigma^2)

In [283]:
#Generate synthetic data
n = 10000000
loc = 5
scale = 3
X = np.random.normal(loc, scale, n)
X_avg = X.mean()
u = loc
S_n = X.var()
print(X_avg - u, S_n)

test_statistic = abs((n - 1)**0.5 * (X_avg - u)/(S_n)**0.5)
test_statistic

-0.0006294275155784845 9.006645727997403


0.6632299998508626

In [88]:
#Compare the test statistics of T test and Z test
#T Test should be larger, i.e. eaiser to reject
test_statistic = abs(n**0.5 * (X_avg - u)/(scale))
test_statistic

1.7692503476278532

In [89]:
alpha = 0.05
q_alpha = stats.t.ppf(1-alpha/2, df = n - 1)
if test_statistic > q_alpha:
  print("Reject H_0")
else:
  print("Failed to reject H_0")

Failed to reject H_0


In [226]:
#Repeat the test for 100 times
#Expect to reject H_0 about 5 times
trials = 100
alpha = 0.05
n = 10000
q_alpha = stats.t.ppf(1-alpha/2, df = n - 1)
loc = 5
scale = 3
S_n = X.var()
results = []

for _ in range(trials):
  X = np.random.normal(loc, scale, n)
  X_avg = X.mean()
  test_statistic = abs((n - 1)**0.5 * (X_avg - u)/(S_n)**0.5)
  results.append(test_statistic > q_alpha)

print(f'After {trials} trials, the null hypothesis is rejected {sum(results)} times at 95% confidence.')

After 100 trials, the null hypothesis is rejected 4 times at 95% confidence.


Two Sample Test (Large Sample Case and known variance)

In [341]:
#Generate 2 independent samples
n = 10000000
m = 1000000
ux = 4
uy = 4
sigma_x = ux**0.5
sigma_y = uy**0.5
X = np.random.normal(ux, 1, n)
Y = np.random.normal(uy, 1, m)
print(X.mean(), Y.mean())
#normalized mean x and mean y


test_statistic = abs(n**0.5 *(X.mean() - Y.mean())/(1 + m/n)**0.5)
test_statistic

4.00001686171669 3.999710351170473


0.9241640691701796

In [342]:
alpha = 0.05
q_alpha = stats.norm.ppf(1-alpha/2)
print(q_alpha)
if test_statistic > q_alpha:
  print("Reject H_0")
else:
  print("Failed to reject H_0")

1.959963984540054
Failed to reject H_0


In [362]:
#Repeat the test for many times
trials = 100
alpha = 0.05
n = 1000
m = 100000
ux = 4
uy = 4

q_alpha = stats.norm.ppf(1-alpha/2)
results = []

for _ in range(trials):
  X = np.random.normal(ux, 1, n)
  Y = np.random.normal(uy, 1, m)
  test_statistic = abs(n**0.5 *(X.mean() - Y.mean())/(1 + m/n)**0.5)
  results.append(test_statistic > q_alpha)

print(f'After {trials} trials, the null hypothesis is rejected {sum(results)} times at 95% confidence.')

After 100 trials, the null hypothesis is rejected 0 times at 95% confidence.


Two Sample T Test (Unknown and unequal variances) \\
Assumption: Samples are normal distributed

$\chi^2$ Test